In [3]:
#restart till get t4 or p4
!nvidia-smi

Tue Apr 27 19:23:47 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
# !pip install wandb
# import wandb
# wandb.init()

In [5]:
# #get 25 gb if necessary
# https://towardsdatascience.com/double-your-google-colab-ram-in-10-seconds-using-these-10-characters-efa636e646ff

In [6]:
# sources
#https://colab.research.google.com/github/huggingface/blog/blob/master/notebooks/01_how_to_train.ipynb#scrollTo=G-kkz81OY6xH
#https://www.kaggle.com/samarthagarwal23/bert-lm-fine-tune-using-gpu-on-wikitext
#https://modelzoo.co/model/pytorch-pretrained-bert#Fine-tuning-with-BERT-running-the-examples
#https://towardsdatascience.com/how-to-use-bert-from-the-hugging-face-transformer-library-d373a22b0209
#https://colab.research.google.com/github/huggingface/blog/blob/master/notebooks/01_how_to_train.ipynb#scrollTo=LTXXutqeDzPi


In [7]:
pip install transformers

     |████████████████████████████████| 2.1MB 9.4MB/s 
     |████████████████████████████████| 901kB 34.9MB/s 
     |████████████████████████████████| 3.3MB 30.0MB/s 


In [8]:
from transformers import BertTokenizer, BertForMaskedLM
from torch.nn import functional as F
import torch

#a szavak tokenizálója
tokenizer = BertTokenizer.from_pretrained("SZTAKI-HLT/hubert-base-cc")
model = BertForMaskedLM.from_pretrained("SZTAKI-HLT/hubert-base-cc")

Some weights of the model checkpoint at SZTAKI-HLT/hubert-base-cc were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Transfer learning

In [10]:
model.train()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32001, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [12]:
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

True

In [13]:
model.num_parameters()
#110 millio!!

110651649

Prepare input

In [14]:
# !wget -c https://cdn-datasets.huggingface.co/EsperBERTo/data/oscar.eo.txt

In [16]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    # file_path="./oscar.eo.txt",
    file_path="./corpustext_openrefined_and_custom_and_paragraph_v20.txt",
    block_size=128,
)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:124: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


CPU times: user 57.9 s, sys: 178 ms, total: 58 s
Wall time: 58 s


In [17]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [18]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./Myfinetuded",
    overwrite_output_dir=True,
    num_train_epochs=15,
    per_gpu_train_batch_size=128,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [19]:
%%time
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Step,Training Loss
500,2.278000
1000,2.128200
1500,2.109500


CPU times: user 48min 42s, sys: 29min 46s, total: 1h 18min 28s
Wall time: 41min 1s


TrainOutput(global_step=1500, training_loss=2.171903971354167, metrics={'train_runtime': 2460.7986, 'train_samples_per_second': 0.61, 'total_flos': 1.616455942236288e+16, 'epoch': 15.0, 'init_mem_cpu_alloc_delta': 2451886080, 'init_mem_gpu_alloc_delta': 443394560, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 13930496, 'train_mem_gpu_alloc_delta': 7519232, 'train_mem_cpu_peaked_delta': 81920, 'train_mem_gpu_peaked_delta': 8540362752})

save the model

In [20]:
#save model
trainer.save_model("./Myfinetuded")

check if mask does work

In [21]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./Myfinetuded",
    tokenizer=BertTokenizer.from_pretrained("SZTAKI-HLT/hubert-base-cc")
)

In [22]:
#test
fill_mask("A tengeri hajózás" + tokenizer.mask_token +".")

[{'score': 0.13542285561561584,
  'sequence': 'A tengeri hajózás veszélyes.',
  'token': 8166,
  'token_str': 'v e s z é l y e s'},
 {'score': 0.044903405010700226,
  'sequence': 'A tengeri hajózás biztonságos.',
  'token': 7928,
  'token_str': 'b i z t o n s á g o s'},
 {'score': 0.01925860345363617,
  'sequence': 'A tengeri hajózás luxus.',
  'token': 11906,
  'token_str': 'l u x u s'},
 {'score': 0.016945049166679382,
  'sequence': 'A tengeri hajózás népszerű.',
  'token': 5468,
  'token_str': 'n é p s z e r ű'},
 {'score': 0.016083816066384315,
  'sequence': 'A tengeri hajózás csodálatos.',
  'token': 8343,
  'token_str': 'c s o d á l a t o s'}]